# Inspect postproces for pe_present_on_image -> pe_exam

tested by exp010, fold0 validation

In [9]:
%cd ~/kaggle/rsna-str/workdir

/home/haito/kaggle/rsna-str/workdir


In [5]:


import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

from src.factory import *
from src.utils import *

In [1]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")
pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

/home/haito/kaggle/rsna-str/workdir


NameError: name 'Path' is not defined

In [5]:
t = train[train.fold == 0]

In [18]:
studies = t.StudyInstanceUID.unique()


In [33]:
agg = t.groupby("StudyInstanceUID")["SOPInstanceUID"].apply(list)
agg_one = t.groupby("StudyInstanceUID").first()

In [84]:
def post1(p_arr, q=90):
    return np.percentile(p_arr, q=q)

In [67]:
from sklearn.metrics import log_loss

In [115]:
### exp 010
p = "output/010_pe_pos/out.valid.fold0"  # ep0
# p = "output/010_pe_pos/out.valid.fold0_ep1"

res = load_pickle(p)

raw_pred = pd.DataFrame({
    "sop": res["ids"],
    "pe_present": res["outputs"]["pe_present_on_image"]
})
raw_pred = raw_pred.set_index("sop")

In [111]:
# 001
_path = "output/001_base/fold0_valid.pickle"
res = load_pickle(_path)
raw_pred = pd.DataFrame({
    "sop": res["ids"],
    "pe_present": res["outputs"]
})
raw_pred = raw_pred.set_index("sop")

In [ ]:
""" exp 001 model (ep1)
q=90.00 logloss:0.4335881782152035
q=91.00 logloss:0.4261582133852183
q=92.00 logloss:0.41820680933855464
q=93.00 logloss:0.40965706920377787
q=94.00 logloss:0.40250102547323763
q=95.00 logloss:0.399617003382453   <=== たまたま Best
q=96.00 logloss:0.4014378415196566
q=97.00 logloss:0.40659142078878496
q=98.00 logloss:0.4293089602081556
q=99.00 logloss:0.4881203309768597
"""

In [ ]:
""" exp 010 model (ep0)
q=90 logloss:0.580963053992852
q=91 logloss:0.5561516892147172
q=92 logloss:0.5302883445027334
q=93 logloss:0.5057102176781858
q=94 logloss:0.48133613115138896
q=95 logloss:0.4547570238934727   <=========== Bad
q=96 logloss:0.4293633715604374
q=97 logloss:0.40440298977917516
q=98 logloss:0.383646086091538
q=98.80 logloss:0.37546814335089845
q=98.90 logloss:0.3744332339894111   <=========== Mininum for fold0
q=99.00 logloss:0.3745150839621268
q=99 logloss:0.374515083962127
q=99.10 logloss:0.3755236864711825
q=99.20 logloss:0.3776048971290255
q=99.5 logloss:0.389937762524077
q=99.7 logloss:0.40671684673171016
q=99.9 logloss:0.43696034136869655
q=99.95 logloss:0.44722869414163663
q=99.99 logloss:0.4569151069554547
"""

In [ ]:
""" exp 010 model (ep1)
q=90.00 logloss:0.5279595949554328
q=91.00 logloss:0.5052608242921037
q=92.00 logloss:0.4834110695747726
q=93.00 logloss:0.46385352130528656
q=94.00 logloss:0.4473139354706962
q=95.00 logloss:0.43102377556895444
q=96.00 logloss:0.41591824329556415
q=97.00 logloss:0.40408809195191403   <================ Best. epが進むほど最適なpercentilがさがる. epが進むとOver-confになるからだと思われる. exp001,best 0.3996
q=98.00 logloss:0.4081253578858078
q=99.00 logloss:0.4430865048629079
"""

In [116]:
#for q in [97,97.5,98, 98.5,99,99.5]:
for q in np.arange(98.5, 99.3, 0.1):
#for q in np.arange(90.0, 99.9, 1):

    LABELS = []
    PREDS = []
    for study in t.StudyInstanceUID.unique():
        sops = agg.loc[study]
        label = agg_one.loc[study]
        label_is_pe = int((not label.indeterminate) and (not label.negative_exam_for_pe))
        LABELS.append(label_is_pe)

        probs_pe_present = raw_pred.loc[sops]
        pe_prob = post1(probs_pe_present, q=q)
        PREDS.append(pe_prob)

        # print(study, label_is_pe, pe_prob)

    print(f"q={q:.2f} logloss:{log_loss(LABELS, PREDS)}")

q=98.50 logloss:0.3782279181914219
q=98.60 logloss:0.3774932380182334
q=98.70 logloss:0.3766140476251227
q=98.80 logloss:0.37546814335089845
q=98.90 logloss:0.3744332339894111
q=99.00 logloss:0.3745150839621268
q=99.10 logloss:0.3755236864711825
q=99.20 logloss:0.3776048971290255


In [82]:
log_loss(LABELS, PREDS)

0.4547570238934727